# Introduction to Knowledge Graphs

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span>
            <ul>
                <li><span><a href="#What-is-a-knowledge-graph?" data-toc-modified-id="What-is-a-knowledge-graph?-1.1">What is a knowledge graph?</a></span></li>
                <li><span><a href="#Key-Terms" data-toc-modified-id="Key-Terms-1.2">Key Terms</a></span></li>
            </ul>
        </li>
        <li><span><a href="#Connect" data-toc-modified-id="Connect-2">Connect</a></span></li>
        <li><span><a href="#Create" data-toc-modified-id="Create-3">Create</a></span></li>
        <li><span><a href="#Data-Model" data-toc-modified-id="Data-Model-4">Data Model</a></span></li>
        <li><span><a href="#Path-Forward" data-toc-modified-id="Path-Forward-5">Path Forward</a></span></li>
    </ul>
</div>

## Introduction

ArcGIS Knowledge integrates graph and spatial analytics to support data discovery, collaborative investigations, link analysis, and information sharing across organizations. An ArcGIS Enterprise portal configured for ArcGIS Knowledge is required. Find out more at [Configure ArcGIS Knowledge](https://enterprise.arcgis.com/en/knowledge/latest/introduction/configure-arcgis-knowledge.htm)

In order to use the [`arcgis.graph`](https://developers.arcgis.com/python/api-reference/arcgis.graph.html) capabilities in the ArcGIS API for Python, you must have an existing knowledge graph. Learn more about [creating knowledge graphs](https://pro.arcgis.com/en/pro-app/latest/help/data/knowledge/create-an-investigation-in-a-project.htm).

### What is a knowledge graph?
A knowledge graph consists of entities and the relationships between them. Each of these entities and relationships can also have properties that contain information describing the entity or relationship. Entities in the model represent real-world objects, concepts, or events. Relationships in the model express how entities are associated with each other. 

### Key Terms
A few key terms to understand are:
- Knowledge Graph: A network that connects people, places, things, and events with each other through relationships.
- Data Model: Defines the types of entities and relationships that can exist in the knowledge graph along with their properties.
- Entity: Representation of a specific thing in the graph such as a person, building, vehicle, etc. These can have any number of properties and be either spatial or nonspatial.
- Relationship: Connects two entities in the graph and represents their relationship such as knows, works_at, drives, etc. These can have any number of properties and are nonspatial.
- Properties: key-value pairs that describe the entity or relationships such as name, start_date, make, model, etc.

## Connect

Using the ArcGIS API for Python, you can connect to a `KnowledgeGraph` with `arcgis.graph`:

In [ ]:
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph

url = "https://myHostName.domain.com/server/rest/services/Hosted/myServiceName/KnowledgeGraphServer"
gis = GIS("home")
knowledge_graph = KnowledgeGraph(url=url, gis=gis)

A knowledge graph service is composed of two primary resources, the data and data model. The data contains entities and relationships with their associated properties. 

## Create
Using the ArcGIS API for Python, you can create a new Knowledge Graph using [create_service()](https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#arcgis.gis.ContentManager.create_service).

In [ ]:
gis = GIS("home")
result = gis.content.create_service(
    name="myKnowledgeGraph",
    capabilities="Query,Editing,Create,Update,Delete",
    service_type="KnowledgeGraph",
)

Services can also be created using create parameters to choose more service options.

In [ ]:
result = gis.content.create_service(
    name="",
    service_type="KnowledgeGraph",
    create_params={
        "name": "myKnowledgeGraph",
        "capabilities": "Query",
        "jsonProperties": {
            "supportsProvenance": False,
            "arcgisManagedData": False,
            "dataSourceItemID": "f63a6725058c4c05a63480521acffe01",
        },
    },
)

Finally, the result from create_service can be used to initialize the KnowledgeGraph.

In [ ]:
knowledge_graph = KnowledgeGraph(url=result.url, gis=gis)

## Data Model

The data model defines the schema of the knowledge graph by defining the entity types and relationship types in the graph:

In [ ]:
knowledge_graph.query_data_model(as_dict=False)

The response from `query_data_model()` has many useful components that describe the schema of the knowledge graph. When `as_dict` is set to `False`, the response will be a `GraphDataModel`. Important information to look for on the data model includes spatial reference, strictness, whether the service is arcgis managed, information on the unique identifier property, and search indexes. The data model also includes detailed information on each entity and relationship type.

```
GraphDataModel(
    data_model_timestamp=1737573982389, 
    spatial_reference={'wkid': 4326}, 
    entity_types=[
        EntityType(
            name='Vehicle', 
            alias='', 
            role='esriGraphNamedObjectRegular', 
            strict=False, 
            properties=[
                GraphProperty(name='objectid', alias='objectid', domain='', field_type='esriFieldTypeOID', geometry_type=None, has_z=False, has_m=False, default_value=None, nullable=False, visible=True, editable=False, required=True, is_system_maintained=True, role='esriGraphPropertyRegular'), 
                GraphProperty(name='globalid', alias='globalid', domain='', field_type='esriFieldTypeGlobalID', geometry_type=None, has_z=False, has_m=False, default_value=None, nullable=False, visible=True, editable=False, required=True, is_system_maintained=True, role='esriGraphPropertyRegular'), 
                GraphProperty(name='make', alias='', domain='', field_type='esriFieldTypeString', geometry_type=None, has_z=False, has_m=False, default_value=None, nullable=True, visible=True, editable=True, required=False, is_system_maintained=False, role='esriGraphPropertyRegular')
            ], 
            field_indexes=[
                FieldIndex(name='esri__oid_idx', is_ascending=True, is_unique=True, fields=['objectid']), 
                FieldIndex(name='esri__globalid_idx', is_ascending=True, is_unique=True, fields=['globalid'])
            ]
        )
    ],
    search_indexes=[
        SearchIndex(
            name='esri__search_idx', 
            supported_category='both_entity_relationship', 
            analyzers=[SearchAnalyzer(name='text_en'), 
            SearchAnalyzer(name='identity')], 
            search_properties={
                'Document': SearchIndexProperties(property_names=['metadata', 'keywords', 'fileExtension', 'globalid', 'name', 'text', 'title', 'contentType', 'url']), 
                'HasDocument': SearchIndexProperties(property_names=['globalid']), 
                'Vehicle': SearchIndexProperties(property_names=['globalid', 'make'])
            }
        )
    ], 
    provenance_source_type_values=ProvenanceSourceTypeValues(value_behavior_array=[]), 
    constraint_rules=[
        RelationshipExclusionRule(name='HasDocumentRule', alias="Relationships of type 'HasDocument' can only have a destination entity of type 'Document'.", disabled=False, role='esriGraphConstraintRuleRoleHasDocument', type='esriGraphRelationshipExclusionRuleType', origin_entity_types=SetOfNamedTypes(set=[], set_complement=[]), relationship_types=SetOfNamedTypes(set=['HasDocument'], set_complement=[]), destination_entity_types=SetOfNamedTypes(set=[], set_complement=['Document']))
    ]
)
```

Entity and relationship types each have a format very similar to each other including basic information about the type like the name, role and strictness as well as properties and field indexes on the type. For a single type, this could look like:
```
EntityType(
    name='Vehicle', 
    alias='', 
    role='esriGraphNamedObjectRegular', 
    strict=False, 
    properties=[
        GraphProperty(name='objectid', alias='objectid', domain='', field_type='esriFieldTypeOID', geometry_type=None, has_z=False, has_m=False, default_value=None, nullable=False, visible=True, editable=False, required=True, is_system_maintained=True, role='esriGraphPropertyRegular'), 
        GraphProperty(name='globalid', alias='globalid', domain='', field_type='esriFieldTypeGlobalID', geometry_type=None, has_z=False, has_m=False, default_value=None, nullable=False, visible=True, editable=False, required=True, is_system_maintained=True, role='esriGraphPropertyRegular'), 
        GraphProperty(name='make', alias='', domain='', field_type='esriFieldTypeString', geometry_type=None, has_z=False, has_m=False, default_value=None, nullable=True, visible=True, editable=True, required=False, is_system_maintained=False, role='esriGraphPropertyRegular')
    ], 
    field_indexes=[
        FieldIndex(name='esri__oid_idx', is_ascending=True, is_unique=True, fields=['objectid']), 
        FieldIndex(name='esri__globalid_idx', is_ascending=True, is_unique=True, fields=['globalid'])
    ]
)
```

## Path Forward

The knowledge graph can be used to <a href="part2-search-query-knowledge-graph.ipynb">search and query a knowledge graph</a> as well as <a href="part3-edit-knowledge-graph.ipynb">add, update and delete</a> types and instances of the data. These will help you to create and use your knowledge graphs.